In [ ]:
class Visit(models.Model):
    date = models.DateField()
    member_id = models.CharField(max_length=15)
    driver = models.CharField(max_length=50) # driver email
    
    def as_tuple(self):
        return (self.driver, self.member_id)

    def __str__(self):
        return f'{self.driver} visited {self.member_id} on {self.date}'

In [1]:
from typing import Dict
from pathlib import Path
import pandas as pd
from simple_jsonifier import make_json, dump_json

In [2]:
def change_keys(dictionary: Dict[str,str]) -> dict:
    return{key.lower().replace('#', 'num').replace(' ', '_'): dictionary[key] 
           for key in dictionary.keys()}

def normalize_string(chars: str) -> str:
    return chars.lower().replace(' ', '_')

In [8]:
df = pd.read_csv(Path(Path.home(), 'desktop', 'DELIVERIES.csv'))
df['Delivery Date'] = pd.to_datetime(df['Delivery Date'])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
completed = df[df['Delivery Status'] == 'Completed'].copy()

In [12]:
emails = {
    'Dylan Burke': 'dburke@gcfb.org',
    'Nate Mills': 'nmills@gcfb.org',
    'Scarlet Decollibus': 'sdecollibus@gcfb.org',
    'Kevin Lomeli': 'klomeli@gcfb.org',
    'Lucy Nord': 'lnord@gcfb.org',
    'Mike Vickers': 'mvickers@gcfb.org',
}


def apply_dict(dictionary):
    return lambda val: dictionary.get(val, val)


visits = (completed[['Member ID', 'Delivery Date', 'Driver']]
              .rename(columns=normalize_string)
              .dropna()
              .applymap(apply_dict(emails)))

visits['delivery_date'] = visits['delivery_date'].dt.strftime('%Y-%m-%d')

In [13]:
dump_json(make_json(visits.to_dict(orient='records'), 'routes.visit'), Path('data', 'visits.json'))